In [205]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import regex as re
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx 
from collections import Counter
from fa2 import ForceAtlas2
from DataCleaningFunctions import DataCleaner, DataCollection, DataProcesser
import os
import json

### old code

In [189]:
def collect_movies_artist_data(genre: str, year:str, sub_years:list) -> pd.DataFrame :

    titles = []
    titles_refs = []
    directors = []
    casts = []
    countries = []
    all_years = []
    genres = []

    if genre == 'horror':
        url = f"https://en.wikipedia.org/wiki/List_of_horror_films_of_{year}"
        sub_years = [year]
    else: 
        url = f"https://en.wikipedia.org/wiki/List_of_{genre}_films_of_the_{year}"

    wiki_page = requests.get(url)
    soup = BeautifulSoup(wiki_page.text, 'html.parser')
    all_tables = soup.find_all('table', {'class':'wikitable'})
            
    for sub_year_idx, sub_year in enumerate(sub_years):   
        for row in all_tables[sub_year_idx].find_all('tr'): 
            columns = row.find_all('td')
            
            # Extracting the subyear of the tables
            if (genre == 'thriller') or (genre == 'fantasy') or (genre=='science_fiction'): 
                if (len(columns)!=0): 
                    if (re.search(r"2\d{3}\n", columns[0].text)):
                        sub_year = columns[0].text[:4]

            if len(columns) >= 4:
                if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
                    try: 
                        title = row.find('a').get_text(strip=True)
                    except: 
                        title = row.find('i').get_text(strip=True)
                    director = columns[0].get_text(strip=True)
                    cast_list = columns[1].get_text(strip=True)
                    country = columns[2].get_text(strip=True)
                elif genre == 'comedy' and sub_year=='2007':       
                    title = row.find('a').get_text(strip=True)
                    director = columns[0].get_text(strip=True)
                    cast_list = columns[1].get_text(strip=True)
                    country = columns[2].get_text(strip=True)
                else: 
                    title = columns[0].get_text(strip=True)
                    director = columns[1].get_text(strip=True)
                    cast_list = columns[2].get_text(strip=True)
                    country = columns[3].get_text(strip=True)

                titles.append(title)
                directors.append(director)
                casts.append(cast_list)
                countries.append(country)
                genres.append(genre)
                all_years.append(sub_year)

            for table in all_tables:
                rows = table.find_all('tr')

                for row in rows:
                    if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
                        first_column = row.find('th')

                    else:
                        first_column = row.find('td')

                    if first_column:
                        first_link = first_column.find('a')
                        title = first_column.get_text(strip=True)

                        if first_link:
                            href = first_link.get('href')
                            href = href.replace('/wiki/', '')
                        else:
                            href = ''

                        titles_refs.append((title, href))


    data = {
    "Title": titles,
    "Director": directors,
    "Cast": casts,
    "Country": countries,
    "Genre": genres, 
    "Year": all_years}

    df = pd.DataFrame(data)

    #if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
    #    df['Hyperref'] = df['Director']
    #else:
    #    df['Hyperref'] = df['Title'].apply(lambda title: next((href for t, href in titles_refs if t == title), ''))

    return df

In [203]:
titles_refs = []
url = f"https://en.wikipedia.org/wiki/List_of_horror_films_of_2009"
wiki_page = requests.get(url)
genre = 'horror'
sub_year = '2009'
soup = BeautifulSoup(wiki_page.text, 'html.parser')
all_tables = soup.find_all('table', {'class':'wikitable'})

for table in all_tables:
    rows = table.find_all('tr')

    for row in rows:
        if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
            first_column = row.find('th')

        else:
            first_column = row.find('td')

        if first_column:
            first_link = first_column.find('a')
            title = first_column.get_text(strip=True)

            if first_link:
                href = first_link.get('href')
                href = href.replace('/wiki/', '')
            else:
                href = ''

            titles_refs.append((title, href))

In [59]:
params = {
        "thriller": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],[[]]]}, 
        "action": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]},             
        "comedy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007"]]},
        "adventure": {"years": ["2010s", "2000s"],
                "sub_years": [["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019", "2020", "2021", "2022"], ["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]}, 
        "horror": {"years": ["2020", "2021", "2022", "2010","2011","2012","2013","2014","2015","2016","2017","2018","2019", "2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"], 
                "sub_years": [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]},
        "fantasy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        "science_fiction": {"years": ["2020s", "2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        }

data_movie_artist = pd.DataFrame()
for genre in params.keys():
    for years, sub_years in zip(params[genre]["years"],params[genre]["sub_years"]):
        data = collect_movies_artist_data(genre, years, sub_years)
        data_movie_artist = pd.concat([data_movie_artist, data]) 

data_movie_artist = data_movie_artist.reset_index(drop=True)


In [60]:
empty_hyperrefs_count = (data_movie_artist['Hyperref'] == '').sum()
print(f"Number of rows with empty Hyperrefs: {empty_hyperrefs_count}")


Number of rows with empty Hyperrefs: 1821


In [89]:
desired_title = "Blind Turn"
row = data_movie_artist.loc[data_movie_artist['Title'] == desired_title]
row

,Title,Director,Cast,Country,Genre,Year,Hyperref
147,Blind Turn,Robert Orr,"Rachel Boston,Jay Dee Walters,John Gabriel Rod...",United States,thriller,2011,/w/index.php?title=Blind_Turn&action=edit&redl...


### New code

In [ ]:

class DataCollection:    
    def collect_movies_artist_data(self, genre: str, year:str, sub_years:list) -> pd.DataFrame:
        titles = []
        directors = []
        casts = []
        countries = []
        all_years = []
        genres = []
        link_refs = []

        # Defining the url. Notice that it is different for the genre horror. 
        if genre == 'horror':
            url = f"https://en.wikipedia.org/wiki/List_of_horror_films_of_{year}"
            sub_years = [year]
        else: 
            url = f"https://en.wikipedia.org/wiki/List_of_{genre}_films_of_the_{year}"

        # Get data using BeautifulSoup and HTML
        wiki_page = requests.get(url)
        soup = BeautifulSoup(wiki_page.text, 'html.parser')
        # Since the movies and artists are stored in tables, we use BeautifulSoup to find all tables in the wiki page
        all_tables = soup.find_all('table', {'class':'wikitable'})

        # The table is divided into subtables for each subyear of year degree.         
        for sub_year_idx, sub_year in enumerate(sub_years):
            for row in all_tables[sub_year_idx].find_all('tr'): 
                columns = row.find_all('td')

                # Extracting the subyear of the tables that are merged and not divided into subtables. 
                if (genre == 'thriller') or (genre == 'fantasy') or (genre=='science_fiction'): 
                    if (len(columns)!=0): 
                        if (re.search(r"2\d{3}\n", columns[0].text)):
                            sub_year = columns[0].text[:4]

                if len(columns) >= 4:
                    if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
                        try: 
                            title = row.find('a').get_text(strip=True)
                        except: 
                            title = row.find('i').get_text(strip=True)
                        director = columns[0].get_text(strip=True)
                        cast_list = columns[1].get_text(strip=True)
                        country = columns[2].get_text(strip=True)
                    elif genre == 'comedy' and sub_year=='2007':       
                        title = row.find('a').get_text(strip=True)
                        director = columns[0].get_text(strip=True)
                        cast_list = columns[1].get_text(strip=True)
                        country = columns[2].get_text(strip=True)
                    else: 
                        title = columns[0].get_text(strip=True)
                        director = columns[1].get_text(strip=True)
                        cast_list = columns[2].get_text(strip=True)
                        country = columns[3].get_text(strip=True)

                    # Getting hyperlinks to plots of movie
                    if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
                        first_column = row.find('th')
                    else:
                        first_column = row.find('td')

                    if first_column: 
                        href_link = first_column.find('a')
                        if href_link:
                            href = href_link.get('href').replace('/wiki/', '')
                        else:
                            href = ''
                    else: 
                        href = ''
            
                    titles.append(title)
                    directors.append(director)
                    casts.append(cast_list)
                    countries.append(country)
                    genres.append(genre)
                    all_years.append(sub_year)
                    link_refs.append(href)
        
        data = {"Title": titles,
                "Director": directors,
                "Cast": casts,
                "Country": countries,
                "Genre": genres, 
                "Year": all_years, 
                "Hyperref": link_refs}
        
        df = pd.DataFrame(data)
        return df

In [ ]:
titles_refs = []
url = f"https://en.wikipedia.org/wiki/List_of_action_films_of_2000"
wiki_page = requests.get(url)
genre = 'action'
sub_year = '2000'
soup = BeautifulSoup(wiki_page.text, 'html.parser')
all_tables = soup.find_all('table', {'class':'wikitable'})

for table in all_tables:
    rows = table.find_all('tr')

    for row in rows:
        if genre == 'horror' and not (sub_year == '2020' or sub_year == '2021' or sub_year == '2022' or sub_year == '2019'):
            first_column = row.find('th')

        else:
            first_column = row.find('td')

        if first_column:
            first_link = first_column.find('a')
            title = first_column.get_text(strip=True)

            if first_link:
                href = first_link.get('href')
                href = href.replace('/wiki/', '')
            else:
                href = ''

            titles_refs.append((title, href))

In [166]:
params = {
        "thriller": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],[[]]]}, 
        "action": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]},             
        "comedy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [["2020", "2021", "2022"],["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019"],["2000", "2001", "2002","2003","2004","2005","2006","2007"]]},
        "adventure": {"years": ["2010s", "2000s"],
                "sub_years": [["2010", "2011", "2012","2013","2014","2015","2016","2017","2018","2019", "2020", "2021", "2022"], ["2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"]]}, 
        "horror": {"years": ["2020", "2021", "2022", "2010","2011","2012","2013","2014","2015","2016","2017","2018","2019", "2000", "2001", "2002","2003","2004","2005","2006","2007","2008","2009"], 
                "sub_years": [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]},
        "fantasy": {"years": ["2020s","2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        "science_fiction": {"years": ["2020s", "2010s", "2000s"],
                "sub_years": [[[]],[[]],[[]]]}, 
        }

data_collection = DataCollection()
data_movie_artist = pd.DataFrame()
for genre in params.keys():
    for years, sub_years in zip(params[genre]["years"],params[genre]["sub_years"]):
        data = data_collection.collect_movies_artist_data(genre, years, sub_years)
        data_movie_artist = pd.concat([data_movie_artist, data]) 

data_movie_artist = data_movie_artist.reset_index(drop=True)
#data_movie_artist.to_csv('data_movie_artist.csv', index=False)

In [167]:
data_movie_artist.shape

(7415, 7)

In [168]:
data_cleaner = DataCleaner(data_movie_artist)
data_cleaner.clean_columns()
data_movie_artist_cleaned = data_cleaner.data

c:\Users\johan\OneDrive - Danmarks Tekniske Universitet\Skrivebord\Cinemartist-Networks-and-Interactions\DataCleaningFunctions.py:321: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(rows_to_keep).reset_index(drop=True)
c:\Users\johan\OneDrive - Danmarks Tekniske Universitet\Skrivebord\Cinemartist-Networks-and-Interactions\DataCleaningFunctions.py:332: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(rows_to_keep).reset_index(drop=True)
c:\Users\johan\OneDrive - Danmarks Tekniske Universitet\Skrivebord\Cinemartist-Networks-and-Interactions\DataCleaningFunctions.py:356: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.data = self.data.append(rows_to_keep).reset_index(dro

In [169]:
data_movie_artist_cleaned.shape

(5724, 7)

In [170]:
def remove_comments(text):
    return re.sub(r'<!--.*?-->', '', text)

In [171]:
def clean_filename(filename):
    # Replace non-alphanumeric characters (except underscores) with underscores
    return re.sub(r'[^a-zA-Z0-9_]', '_', filename)

In [174]:
data_movie_artist

,Title,Director,Cast,Country,Genre,Year,Hyperref
0,365 Days,barbara biaowas tomasz mandes,"anna-maria sieklucka,michele morrone, bronisaw...",poland,thriller,2020,365_Days_(2020_film)
1,7500,patrick vollrath,"joseph gordon-levitt,aylin tezel",germany austria united states,thriller,2020,7500_(film)
2,Antebellum,bush renz,"janelle monae,eric lange,jena malone,jack hust...",united states,thriller,2020,Antebellum_(film)
3,Archenemy,adam egypt mortimer,"joe manganiello,skylan brooks, zolee griggs,pa...",united states united kingdom,thriller,2020,Archenemy_(film)
4,Asuraguru,a raajdheep,"vikram prabhu,mahima nambiar,subbaraju,yogi babu",india,thriller,2020,Asuraguru
...,...,...,...,...,...,...,...
7410,The Time Traveler s Wife,robert schwentke,"eric bana,rachel mc adams,ron livingston",united states,science_fiction,2009,The_Time_Traveler%27s_Wife_(film)
7411,Transformers Revenge of the Fallen,michael bay,"shia la beouf,megan fox,josh duhamel,tyrese gi...",united states,science_fiction,2009,Transformers:_Revenge_of_the_Fallen
7412,Watchmen,zack snyder,"jackie earle haley,patrick wilson,carla gugino",united states,science_fiction,2009,Watchmen_(film)
7413,Wonder Woman,lauren montgomery,"keri russell,nathan fillion,alfred molina,rosa...",united states,science_fiction,2009,Wonder_Woman_(2009_film)


In [173]:
data_movie_artist_cleaned

,Title,Director,Cast,Country,Genre,Year,Hyperref
0,365 Days,barbara biaowas tomasz mandes,"anna-maria sieklucka,michele morrone, bronisaw...",europe,thriller,2020,365_Days_(2020_film)
1,7500,patrick vollrath,"joseph gordon-levitt,aylin tezel",other,action-thriller,2020,7500_(film)
2,Archenemy,adam egypt mortimer,"joe manganiello,skylan brooks, zolee griggs,pa...",united states united kingdom,action-thriller,2020,Archenemy_(film)
3,Asuraguru,a raajdheep,"vikram prabhu,mahima nambiar,subbaraju,yogi babu",india,action-thriller,2020,Asuraguru
4,Aswathama,ramana teja,"naga shourya,mehreen pirzada",india,action-thriller,2020,Aswathama_(film)
...,...,...,...,...,...,...,...
5719,Renegades,steven quale,"sullivan stapleton,j. k. simmons,charlie bewley",france,action-adventure,2017,Renegades_(2017_film)
5720,Scott Pilgrim vs the World,edgar wright,"michael cera,mary elizabeth winstead,kieran cu...",united states,action-comedy,2010,Scott_Pilgrim_vs._the_World
5721,Skin Trade,ekachai uekrongtham,"dolph lundgren,tony jaa,michael jai white,ron ...",united states,action-thriller,2014,Skin_Trade_(film)
5722,The League of Extraordinary Gentlemen,stephen norrington,"sean connery,shane west,stuart townsend",united states,other,2003,The_League_of_Extraordinary_Gentlemen_(film)


In [178]:

def get_movie_plots(data_movie_artist):
    no_plot_counter = 0

    for index, row in data_movie_artist.iterrows():
        wikiname = row['Hyperref']
        genre = row['Genre']
        year = row['Year']

        # Thriller
        if any(data_movie_artist[data_movie_artist["Title"].isin(["Life Is an Art", "A Morass", "Blind Turn", "Requiem pour une tueuse", "She and She", "American Sunset", "Espion(s)"])]["Hyperref"] == wikiname):
            plot_text = "No plot"
            no_plot_counter += 1
            continue
        
        # Action
        if any(data_movie_artist[data_movie_artist["Title"].isin(["Sheenogai", "Vampire Warriors", "Bangkok Revenge", "My Kingdom", "Wild 7", "San Andreas Quake"])]["Hyperref"] == wikiname):
            plot_text = "No plot"
            no_plot_counter += 1
            continue

        # Check if the wikiname contains special characters
        if any(char in wikiname for char in ["'", '%', '&']):
            # Set the title name from the data_movie_artist DataFrame column that matches the Hyperref column
            wikiname = data_movie_artist.loc[data_movie_artist['Hyperref'] == wikiname, 'Title'].values[0]

        try:
            # Make the request
            baseurl = "https://en.wikipedia.org/w/api.php?"  # the wiki API
            params = {
                "action": "query",
                "prop": "revisions",
                "rvprop": "content",
                "format": "json",
                "titles": wikiname
            }

            print(wikiname)
            wikitext = requests.get(baseurl, params=params)
            wikijson = wikitext.json()
            page_id = next(iter(wikijson["query"]["pages"]))
            # Check if "revisions" key is present
            revisions = wikijson["query"]["pages"][page_id].get("revisions", [])
            if not revisions:
                raise KeyError("No revisions key")

            wiki_content = revisions[0]["*"]
            wiki_content_without_comments = remove_comments(wiki_content)
            plot_section_match = re.search(r'==\s*Plot\s*==\s*(.*?)(==|\Z)', wiki_content_without_comments, re.DOTALL | re.IGNORECASE)
            plot_section = plot_section_match.group(1) if plot_section_match else None

            # Extract the plot text
            if plot_section:
                plot_text = remove_comments(plot_section)
                # Remove hyperlinks and display only the link text
                plot_text = re.sub(r'\[\[([^|\]]*?\|)?([^\]]*?)\]\]', r'\2', plot_text)
            else:
                plot_text = "Plot not found."
                no_plot_counter += 1

            # Create folders based on genre and subfolders based on the year
            genre_folder = clean_filename(genre)
            year_folder = clean_filename(str(year))

            os.makedirs(f'{genre_folder}/{year_folder}', exist_ok=True)

            # Save plot text in a .txt file
            with open(os.path.join(genre_folder, year_folder, f'{clean_filename(wikiname)}.txt'), 'w', encoding='utf-8') as f:
                f.write(plot_text)

        except KeyError as e:
            print(f"Error processing {wikiname}: {e}")
            plot_text = "No plot"
            no_plot_counter += 1
            continue
    print(no_plot_counter)


In [183]:
data_movie_artist_cleaned

,Title,Director,Cast,Country,Genre,Year,Hyperref
0,365 Days,barbara biaowas tomasz mandes,"anna-maria sieklucka,michele morrone, bronisaw...",europe,thriller,2020,365_Days_(2020_film)
1,7500,patrick vollrath,"joseph gordon-levitt,aylin tezel",other,action-thriller,2020,7500_(film)
2,Archenemy,adam egypt mortimer,"joe manganiello,skylan brooks, zolee griggs,pa...",united states united kingdom,action-thriller,2020,Archenemy_(film)
3,Asuraguru,a raajdheep,"vikram prabhu,mahima nambiar,subbaraju,yogi babu",india,action-thriller,2020,Asuraguru
4,Aswathama,ramana teja,"naga shourya,mehreen pirzada",india,action-thriller,2020,Aswathama_(film)
...,...,...,...,...,...,...,...
5719,Renegades,steven quale,"sullivan stapleton,j. k. simmons,charlie bewley",france,action-adventure,2017,Renegades_(2017_film)
5720,Scott Pilgrim vs the World,edgar wright,"michael cera,mary elizabeth winstead,kieran cu...",united states,action-comedy,2010,Scott_Pilgrim_vs._the_World
5721,Skin Trade,ekachai uekrongtham,"dolph lundgren,tony jaa,michael jai white,ron ...",united states,action-thriller,2014,Skin_Trade_(film)
5722,The League of Extraordinary Gentlemen,stephen norrington,"sean connery,shane west,stuart townsend",united states,other,2003,The_League_of_Extraordinary_Gentlemen_(film)


In [191]:
data_movie_artist[data_movie_artist['Title'] == 'Griff Furst']

,Title,Director,Cast,Country,Genre,Year,Hyperref


In [186]:
baseurl = "https://en.wikipedia.org/w/api.php?"  # the wiki API
params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "format": "json",
    "titles": 'Wolvesbayne'
}


wikitext = requests.get(baseurl, params=params)
wikijson = wikitext.json()
page_id = next(iter(wikijson["query"]["pages"]))
# Check if "revisions" key is present
revisions = wikijson["query"]["pages"][page_id].get("revisions", [])
if not revisions:
    raise KeyError("No revisions key")

wiki_content = revisions[0]["*"]
wiki_content_without_comments = remove_comments(wiki_content)
plot_section_match = re.search(r'==\s*Plot\s*==\s*(.*?)(==|\Z)', wiki_content_without_comments, re.DOTALL | re.IGNORECASE)
plot_section = plot_section_match.group(1) if plot_section_match else None

# Extract the plot text
if plot_section:
    plot_text = remove_comments(plot_section)
    # Remove hyperlinks and display only the link text
    plot_text = re.sub(r'\[\[([^|\]]*?\|)?([^\]]*?)\]\]', r'\2', plot_text)
else:
    plot_text = "Plot not found."


In [187]:
plot_text

'Following an attack, ruthless businessman Russell Bayne shapeshifts into a werewolf and seeks help from Alex Layton, an occult shop owner. Together, they learn that vampires under the leadership of Von Griem seek to resurrect their queen, Lilith, in order to unite the warring factions. Bayne allies with a vampire hunter, Jacob Van Helsing, and they battle Lilith and her vampire servants.\n\n'

In [179]:
get_movie_plots(data_movie_artist_cleaned)

365_Days_(2020_film)
7500_(film)
Archenemy_(film)
Asuraguru
Aswathama_(film)
Attack_(2020_film)
Ava_(2020_film)
Dead_Reckoning_(2020_film)
Earth_and_Blood
Extraction_(2020_film)
A_Fall_From_Grace
Fatale_(film)
God_Father_(2020_film)
Honest_Thief
Horizon_Line
Malang_(film)
Monster_Hunter_(film)
Paydirt_(film)
Songbird_(2020_film)
Spell_(film)
The_Crimes_That_Bind_(2020_film)
The_Devil_All_the_Time_(film)
The_Last_Days_of_American_Crime
The_Nest_(2020_film)
The_Night_(2020_film)
The_Retreat_(2020_film)
The_Tax_Collector
Unknown_Origins
V_(2020_film)
Walter_(2020_film)
We_Still_Say_Grace
Akshara_(film)
Body_Brokers
Cold_Case_(film)
Copshop
Crime_Story_(2021_film)
Deadly_Illusions
Drishyam_2
Fever_Dream_(film)
Jagame_Thandhiram
Lockdown_(2021_Nigerian_film)
Master_(2021_film)
My_Son_(2021_film)
Net_(film)
Nightmare_Alley_(2021_film)
No_Sudden_Move
No_Time_to_Die
Nobody_(2021_film)
Oxygen_(2021_film)
Ram_(film)
Silk_Road_(film)
Son_(2021_film)
The_Gateway_(2021_film)
The_Hole_in_the_Fence
T

### IMDb extract ratings

In [162]:
import pandas as pd
from imdb import IMDb

# Create a DataFrame with a column of movie titles
movies_df = data_movie_artist[:100]

# Initialize the IMDb object
ia = IMDb()

# Function to get movie rating
def get_movie_rating(title):
    try:
        movie = ia.search_movie(title)[0]  # Assume the first search result is the correct movie
        ia.update(movie)
        return movie.data['rating']
    except Exception as e:
        print(f"Error fetching rating for {title}: {e}")
        return None

# Add a new column to the DataFrame with the ratings
movies_df['Rating'] = movies_df['Title'].apply(get_movie_rating)

# Display the DataFrame
print(movies_df)


Error fetching rating for The Crimes That Bind: 'rating'
Error fetching rating for Ram: 'rating'
Error fetching rating for The Hole in the Fence: 'rating'
Error fetching rating for Monkey Man: 'rating'
Error fetching rating for Salaar: 'rating'
         Title                            Director  \
0     365 Days       Barbara BiałowąsTomasz Mandes   
1         7500                    Patrick Vollrath   
2   Antebellum                         Bush + Renz   
3    Archenemy                 Adam Egypt Mortimer   
4    Asuraguru                         A.Raajdheep   
..         ...                                 ...   
95      Salaar                      Prashanth Neel   
96   Samaritan                        Julius Avery   
97      Scream  Matt Bettinelli-OlpinTyler Gillett   
98     The 355                       Simon Kinberg   
99  The Batman                         Matt Reeves   

                                                 Cast  \
0   Anna-Maria Sieklucka,Michele Morrone, Bronisł

C:\Users\johan\AppData\Local\Temp\ipykernel_30528\503449598.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Rating'] = movies_df['Title'].apply(get_movie_rating)


In [163]:
import pandas as pd
from imdb import IMDb
import time


In [164]:

# Create a DataFrame with a column of movie titles
movies_df = data_movie_artist[:100]

# Initialize the IMDb object
ia = IMDb()

# Function to get movie rating with caching
def get_movie_rating(title, cache={}):
    if title in cache:
        return cache[title]

    try:
        movie = ia.search_movie(title)[0]  # Assume the first search result is the correct movie
        ia.update(movie)
        rating = movie.data.get('rating')
        cache[title] = rating
        return rating
    except Exception as e:
        print(f"Error fetching rating for {title}: {e}")
        return None

# Add a new column to the DataFrame with the ratings
start_time = time.time()
movies_df['Rating'] = movies_df['Title'].apply(get_movie_rating)
end_time = time.time()

# Display the DataFrame
print(movies_df)
print(f"Time taken: {end_time - start_time} seconds")


         Title                            Director  \
0     365 Days       Barbara BiałowąsTomasz Mandes   
1         7500                    Patrick Vollrath   
2   Antebellum                         Bush + Renz   
3    Archenemy                 Adam Egypt Mortimer   
4    Asuraguru                         A.Raajdheep   
..         ...                                 ...   
95      Salaar                      Prashanth Neel   
96   Samaritan                        Julius Avery   
97      Scream  Matt Bettinelli-OlpinTyler Gillett   
98     The 355                       Simon Kinberg   
99  The Batman                         Matt Reeves   

                                                 Cast  \
0   Anna-Maria Sieklucka,Michele Morrone, Bronisła...   
1                    Joseph Gordon-Levitt,Aylin Tezel   
2   Janelle Monáe,Eric Lange,Jena Malone,Jack Hust...   
3   Joe Manganiello,Skylan Brooks, Zolee Griggs,Pa...   
4    Vikram Prabhu,Mahima Nambiar,Subbaraju,Yogi Babu   
..       

C:\Users\johan\AppData\Local\Temp\ipykernel_30528\256459305.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Rating'] = movies_df['Title'].apply(get_movie_rating)


### Next section
_______

In [94]:
import requests
import re

In [263]:
# Some cast are empty so we need to remove those movies: 
data_movie_artist = data_movie_artist[data_movie_artist['Cast'] != ''].reset_index(drop=True)

# Movies that exists in two different years cleaning: 
# I did some research and found out that they exists in these years: 
#data_movie_artist[data_movie_artist['Title']=='Run Sweetheart Run'] # findes kun i 2020
#data_movie_artist[data_movie_artist['Title']=='The Black Phone'] # findes kun i 2021
#data_movie_artist[data_movie_artist['Title']=='Bhool Bhulaiyaa 2' ]# findes kun i 2022
#data_movie_artist[data_movie_artist['Title']=='Apartment 143' ]# findes kun i 2011
#data_movie_artist[data_movie_artist['Title']=='Underworld: Blood Wars' ]# findes kun i 2016

movies_to_remove = {
    'Run Sweetheart Run': "2022",
    'The Black Phone': "2022",
    'Bhool Bhulaiyaa 2': "2021",
    'Apartment 143': "2012",
    'Underworld: Blood Wars': "2017", 
    'Bloody Bloody Bible Camp': '2012'}
for title, year in movies_to_remove.items():
    mask = data_movie_artist[(data_movie_artist['Title'] == title) & (data_movie_artist['Year'] == year)]
    data_movie_artist = data_movie_artist.drop(mask.index[0]).reset_index(drop=True)


# Different Movies but same title
duplicates_in_title = data_movie_artist[data_movie_artist.duplicated(subset=['Title'], keep=False)]
filtered_duplicates = duplicates_in_title[~duplicates_in_title.duplicated(subset=['Director', 'Country'], keep=False)]
for index, row in filtered_duplicates.iterrows():
    data_movie_artist.loc[index, 'Title'] = row['Title'] + ' ' + row['Director']
data_movie_artist = data_movie_artist.reset_index(drop=True)


### Same movie different Genre
duplicates = data_movie_artist[data_movie_artist[['Title', 'Country', 'Year']].duplicated(keep=False)]

genre_mapping = {}
for index, row in duplicates.iterrows():
    key = (row['Title'], row['Country'], row['Year'])
    if key not in genre_mapping:
        genre_mapping[key] = set()
    genre_mapping[key].add(row['Genre'])

# Create a boolean mask for duplicates
is_duplicate = data_movie_artist[['Title', 'Country', 'Year']].duplicated(keep=False)

data_movie_artist.loc[is_duplicate, 'Genre'] = data_movie_artist[is_duplicate].apply(
    lambda row: 'action-horror' if 'horror' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'action' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('action-thriller' if 'thriller' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'action' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('comedy-action' if 'action' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'comedy' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('comedy-horror' if 'horror' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'comedy' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else 
        ('drama-thriller' if 'thriller' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'drama' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('drama-comedy' if 'drama' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'comedy' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('drama-action' if 'drama' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'action' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('action-adventure' if 'adventure' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'action' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('thriller-horror' if 'thriller' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'horror' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('comedy-thriller' if 'thriller' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'comedy' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('comedy-adventure' if 'adventure' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'comedy' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('thriller-adventure' if 'adventure' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'thriller' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
        ('drama-horror' if 'drama' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'horror' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
         ('adventure-horror' if 'adventure' in genre_mapping[(row['Title'], row['Country'], row['Year'])] and 'horror' in genre_mapping[(row['Title'], row['Country'], row['Year'])] else
         row['Genre']))))))))))))), axis=1)

data_movie_artist = data_movie_artist.drop_duplicates(subset=['Title', 'Country', 'Year', 'Genre']).reset_index(drop=True)

# Movies which exists beceuase of difference in country names  
data_movie_artist['Country_Length'] = data_movie_artist['Country'].str.len()
duplicate_rows = data_movie_artist[data_movie_artist.duplicated(subset=['Title', 'Director'], keep=False)]
filtered_data = duplicate_rows.sort_values(by='Country_Length').drop_duplicates(subset=['Title', 'Director'], keep='first').drop(columns=['Country_Length'])
data_movie_artist = pd.concat([data_movie_artist.drop(duplicate_rows.index), filtered_data]).reset_index(drop=True).drop(columns=['Country_Length'])

In [264]:
data_movie_artist['Country'].value_counts().reset_index()

,index,Country
0,United States,3518
1,Japan,218
2,United Kingdom,197
3,France,145
4,South Korea,115
...,...,...
438,United KingdomTanzania,1
439,United KingdomSpainBelgium,1
440,United states,1
441,DenmarkUnited KingdomNorwaySwedenGermany,1


In [265]:
data_movie_artist['Genre'].value_counts()

horror                1883
comedy                1021
science_fiction        633
action                 596
adventure              463
thriller               429
fantasy                417
action-thriller        140
comedy-action           86
action-adventure        82
comedy-horror           56
action-horror           56
thriller-horror         40
comedy-adventure        30
thriller-adventure      10
adventure-horror         5
Name: Genre, dtype: int64

In [64]:
duplicates_in_title = data_movie_artist[data_movie_artist.duplicated(subset=['Title'], keep=False)]
filtered_duplicates = duplicates_in_title[~duplicates_in_title.duplicated(subset=['Director', 'Country'], keep=False)]
for index, row in filtered_duplicates.iterrows():
    data_movie_artist.loc[index, 'Title'] = row['Title'] + ' ' + row['Director']
data_movie_artist = data_movie_artist.reset_index(drop=True)

In [266]:
data_movie_artist[data_movie_artist['Title'].duplicated(keep=False)]

,Title,Director,Cast,Country,Genre,Year
48,Master,Lokesh Kanagaraj,"Vijay,Vijay Sethupathi,Malavika Mohanan,Arjun ...",India,action-thriller,2021
77,Beast,Nelson Dilipkumar,"Vijay,Pooja Hegde",India,action-thriller,2022
87,Master,Mariama Diallo,"Regina Hall,Zoe Renee,Amber Gray,Molly Bernard...",United States,thriller-horror,2022
325,Pulse,Kiyoshi Kurosawa,"Haruhiko Kato,Kumiko Asō,Koyuki",Japan,thriller,2001
503,Pulse,Jim Sonzero,"Kristen Bell,Ian Somerhalder,Rick Gonzalez",United States,thriller-horror,2006
958,Fantastic Four,Josh Trank,"Miles Teller,Kate Mara,Michael B. Jordan,Jamie...",United States,action-adventure,2015
990,Captain America: Civil War,"Antony Russo, Joe Russo","Chris Evans,Robert Downey Jr.",United States,action,2016
1055,Avengers: Infinity War,"Antony Russo, Joe Russo","Robert Downey Jr.,Scarlett Johansson,Chris Pra...",United States,action,2018
1084,Robin Hood,Otto Bathurst,"Taron Egerton,Jamie Foxx,Ben Mendelsohn,Jamie ...",United States,action-adventure,2018
2499,Robin Hood,Ridley Scott,"Russell Crowe,Cate Blanchett,William Hurt,Max ...",United States,adventure,2010


## Split cast into each row

In [235]:
df = data_movie_artist.copy()

In [236]:
df['Cast'] = df['Cast'].str.split(',')
df = df.explode('Cast').reset_index(drop=True)
df['Cast'] = df['Cast'].str.lower()
df['Title'] = df['Title'].str.lower()


In [238]:
# Number of nodes: 
nr_nodes = len(df['Cast'].unique())
print(nr_nodes)

2207


In [241]:
unique_cast = df.groupby('Cast')['Title'].agg(list).reset_index()
unique_cast.rename(columns={'Title': 'Titles'}, inplace=True)

# Step 4
artist_connections = {}
for index, row in df.iterrows():
    current_artist = row['Cast']
    other_artists = df[df['Title'] == row['Title']]['Cast'].tolist()
    other_artists.remove(current_artist)
    
    if current_artist in artist_connections:
        artist_connections[current_artist].extend(other_artists)
    else:
        artist_connections[current_artist] = other_artists

# Remove duplicates in the "connected" list
for artist, connections in artist_connections.items():
    artist_connections[artist] = list(set(connections))

artist_connections_df = pd.DataFrame(list(artist_connections.items()), columns=['Cast', 'connected'])

# Step 5
# Add a new column for counting connected artists
artist_connections_df['connected_count'] = artist_connections_df['connected'].apply(len)

# Merge the DataFrames
final_df = unique_cast.merge(artist_connections_df, on='Cast')


In [243]:
final_df['connected_count'].sum()

11037

In [244]:
artist_connections = {}
for index, row in df.iterrows():
    current_artist = row['Cast']
    other_artists = df[df['Title'] == row['Title']]['Cast'].tolist()
    other_artists.remove(current_artist)  # Remove the current artist from the list
    if current_artist in artist_connections:
        artist_connections[current_artist].extend(other_artists)
    else:
        artist_connections[current_artist] = other_artists

# Convert the dictionary to a DataFrame
artist_connections_df = pd.DataFrame(list(artist_connections.items()), columns=['cast', 'connected'])


In [169]:
final_df = unique_cast.merge(artist_connections_df, on='Cast')

In [178]:
final_df['connected'][final_df['Cast']=='tom cruise']


4969    [rosamund pike, robert duvall, tom cruise, ros...
Name: connected, dtype: object

In [148]:
df['Title'] = df['Title'].str.lower()

In [155]:
df[df[['Title', 'Cast', 'Genre']].duplicated()]

,Title,Director,Cast,Country,Genre,Year
2896,kingsman: the secret service,Matthew Vaughn,colin firth,United StatesUnited Kingdom,action,2015
2898,kingsman: the secret service,Matthew Vaughn,samuel l. jackson,United StatesUnited Kingdom,action,2015
4561,jack goes boating,Philip Seymour Hoffman,amy ryan,United States,comedy,2010
5115,the players,Various,geraldine nakache,France,comedy,2012
7574,sint,Dick Maas,egbert-jan weeber,Netherlands,horror,2010
7817,apartment 143,Carles Torrens,kai lennox,Spain,horror,2012
7818,apartment 143,Carles Torrens,gia mantegna,Spain,horror,2012
7845,bloody bloody bible camp,Vito Trabuco,reggie bannister,United States,horror,2012
7846,bloody bloody bible camp,Vito Trabuco,tim sullivan,United States,horror,2012
7847,bloody bloody bible camp,Vito Trabuco,ron jeremy,United States,horror,2012


In [156]:
df[df['Title'] == 'kingsman: the secret service']


,Title,Director,Cast,Country,Genre,Year
863,kingsman: the secret service,Matthew Vaughn,colin firth,"United Kingdom,United States",thriller,2014
864,kingsman: the secret service,Matthew Vaughn,taron egerton,"United Kingdom,United States",thriller,2014
865,kingsman: the secret service,Matthew Vaughn,samuel l. jackson,"United Kingdom,United States",thriller,2014
2743,kingsman: the secret service,Matthew Vaughn,colin firth,United States,action,2014
2744,kingsman: the secret service,Matthew Vaughn,samuel l. jackson,United States,action,2014
2745,kingsman: the secret service,Matthew Vaughn,mark strong,United States,action,2014
2896,kingsman: the secret service,Matthew Vaughn,colin firth,United StatesUnited Kingdom,action,2015
2897,kingsman: the secret service,Matthew Vaughn,taron egerton,United StatesUnited Kingdom,action,2015
2898,kingsman: the secret service,Matthew Vaughn,samuel l. jackson,United StatesUnited Kingdom,action,2015
2899,kingsman: the secret service,Matthew Vaughn,,United StatesUnited Kingdom,action,2015
